In [1]:
# Library import
 
import pandas as pd
from pathlib import Path
import datetime as dt
import numpy as np
import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

C:\Users\eramirez\AppData\Local\Temp\ipykernel_22056\3170383445.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Rango de análisis

s_FechaIni='2024-02-27'
s_FechaFin='2024-02-27'


# Set data
b_day=1

if b_day==1:
    s_fecha='2024-02-01'
    formato='%Y-%m-%d'
    d_fecha=dt.datetime.strptime(s_fecha,formato)
    print(d_fecha)

else:
    d_fecha=dt.datetime.now()

year=d_fecha.year
month=d_fecha.month
day=d_fecha.day


2024-02-01 00:00:00


In [3]:
# Get main path and other folders
s_mainpath=Path.cwd()
s_parentpath=s_mainpath.parent
s_folderpath='AnalisisMensual'
s_file='Mantenimientos_Transmision.xlsx'
s_filepath=s_parentpath.joinpath(s_folderpath,str(year) + '-' + "{:02d}".format(month),s_file)

print(s_filepath)

c:\Alejo\Eje de Planeación\Análisis Eléctrico\AnalisisMensual\2024-02\Mantenimientos_Transmision.xlsx


In [4]:
def ColumAfec(TipoAfe):
    if TipoAfe=='Desenergizado - (D)':
        TA='D'
    elif TipoAfe=='DesenergizadoAterrizado - (DA)':
        TA='DA'
    elif TipoAfe=='DesenergizadoDespejado - (DD)':
        TA='DD'
    elif TipoAfe=='DesenergizadoParcial - (DP)':
        TA='DP'
    elif TipoAfe=='Traslado De Carga - (TDC)':
        TA='TDC'
    elif TipoAfe=='Demanda No Atendida - (DNA)':
        TA='DNA'
    elif TipoAfe=='Riesgo de Disparo - (RD)':
        TA='RD'
    else:
        TA=TipoAfe

    return TA

In [5]:
#Read file

df_data=pd.read_excel(s_filepath, sheet_name='MmtosCaribe')
i_indmax=df_data.shape[0]
for ind in df_data.index:

      if ind+2<=i_indmax:
            if (pd.isna(df_data.at[ind+1,'Consecutivo'])):
                  df_data.at[ind+1,'Consecutivo']=df_data.at[ind,'Consecutivo']

# Identificar el elmento que es principal
df_data['TipoAfe'].fillna('ELementoPrincipal',inplace=True)

df_data.rename(columns={'TipoAfe':'TipoAfe1'},inplace=True)

df_data['TipoAfe']=df_data.apply(lambda row: ColumAfec(row['TipoAfe1']),axis=1)

df_data.drop('TipoAfe1',axis=1,inplace=True)

# Tipo de afectaación a filtrar
l_Afe=['D', 'DA', 'DD', 'DP', 'TDC', 'DNA']

df_mmto=df_data[(df_data.TipoAfe.isin(l_Afe)) | ((df_data.TipoAfe=='RD') & (df_data.Detalle=='Simultáneo'))]
df_elmmain=df_data[(df_data.TipoAfe=='ELementoPrincipal')][['Consecutivo','Elemento']]
df_elmmain.rename(columns={'Elemento':'ElmPrincipal'},inplace=True)

columnsini=list(df_mmto.columns)
columnsini.append('ElmPrincipal')

df_resmmto=pd.merge(df_mmto,df_elmmain, on='Consecutivo', how='inner')[columnsini]

# Mapeo SIO Digsilent
s_file='Mapeo_SIO_Dig.xlsx'
s_filepath=s_parentpath.joinpath(s_folderpath,s_file)
df_SIO_Dig=pd.read_excel(s_filepath, sheet_name='SIO_Dig')

In [6]:
# Cambiar las fechas del formato original a tipo fecha
df_resmmto['Fechainicial_m']=pd.to_datetime(df_resmmto['Fechainicial'],format='%d/%m/%Y %H:%M')
df_resmmto['Fechafinal_m']=pd.to_datetime(df_resmmto['Fechafinal'],format='%d/%m/%Y %H:%M')

In [7]:
# Funciones para encontrar los periodos iniciales y finales de cada día
def New_ColumnHIni(FechaIni,d_Fecha):
    # Set hour at ini
    if pd.to_datetime(FechaIni.strftime('%Y-%m-%d'))==d_Fecha:
        horaIni=FechaIni.hour+1
    else:
        horaIni=1
    
    return horaIni

def New_ColumnHFin(FechaFin,d_Fecha):
    # Set hour at fin
    if pd.to_datetime(FechaFin.strftime('%Y-%m-%d'))==d_Fecha:
        if FechaFin.minute>1:
            horaFin=FechaFin.hour+1
        else:
            horaFin=FechaFin.hour
    else:
        horaFin=24

    return horaFin

In [8]:
# Función para determinar las aperturas por día
def get_aper_dia(df_mmtoday):

    l_uniqueElm=list(df_mmtoday['Elemento'].unique())

    l_data=[]

    for Elm in l_uniqueElm:

        # Obtener información de cada elemento
        df_UnqElm=df_mmtoday[df_mmtoday.Elemento==Elm]

        l_Elm=[]
        l_TipoAfe=list(df_UnqElm['TipoAfe'].unique())
        s_TipoAfe=''
        for tf in l_TipoAfe:
            if len(s_TipoAfe)==0:
                s_TipoAfe=s_TipoAfe + str(tf)    
            else:
                s_TipoAfe=s_TipoAfe + ', ' + str(tf)

        i_minper=df_UnqElm['Pini'].min()
        i_maxper=df_UnqElm['Pfin'].max()

        l_Elm.append(Elm)
        l_Elm.append(s_TipoAfe)
        l_Elm.append(i_minper)
        l_Elm.append(i_maxper)     

        l_data.append(l_Elm)

    columns=['Elemento','TipoAfe','Pini','Pfin']

    df_Ap_dia=pd.DataFrame(l_data,columns=columns)

    return df_Ap_dia

In [9]:
# Se recorre cada fecha y se identifican las aperturas

#Camniar a formato fecha
d_FechaIni=dt.datetime.strptime(s_FechaIni,formato)
d_FechaFin=dt.datetime.strptime(s_FechaFin,formato)

delta=dt.timedelta(days=1)

d_Fecha=d_FechaIni

s_file='Mmtosdia.xlsx'
s_filepath=s_parentpath.joinpath(s_folderpath,str(year) + '-' + "{:02d}".format(month),s_file)

df_RD_NoMapeo=pd.DataFrame()
df_AP_NoMapeo=pd.DataFrame()
with pd.ExcelWriter(s_filepath,engine='xlsxwriter') as writer:

    while d_Fecha<=d_FechaFin:
        # Encontrar los que aplica para la fecha
        df_mmtoday=df_resmmto[(d_Fecha>=pd.to_datetime(df_resmmto['Fechainicial_m'].dt.strftime('%Y-%m-%d'))) & (d_Fecha<=df_resmmto['Fechafinal_m'])]

        # Encontrar periodo inicial y periodo final de apertura
        df_mmtoday['Pini']=df_mmtoday.apply(lambda row: New_ColumnHIni(row['Fechainicial_m'],d_Fecha),axis=1)
        df_mmtoday['Pfin']=df_mmtoday.apply(lambda row: New_ColumnHFin(row['Fechafinal_m'],d_Fecha),axis=1)

        # Eliminar columnas auxiliares
        df_mmtoday.drop('Fechainicial_m',inplace=True,axis=1)
        df_mmtoday.drop('Fechafinal_m',inplace=True,axis=1)

        # Definir RD simultáneo
        df_RD=df_mmtoday[(df_mmtoday.TipoAfe=='RD')]
        df_RD=pd.merge(df_RD,df_SIO_Dig, left_on='Elemento',right_on='Nombre_SIO', how='left')[['Consecutivo','Elemento','TipoAfe','Detalle','Pini','Pfin','Fkey','Estado','TipoElm']]
        df_RD['Incluir']=1

        # Validar los RD que no se encontraron en el MAPEO
        df_Aux=df_RD[df_RD['Fkey'].isna()][['Consecutivo','Elemento']]
        df_Aux['Fecha']=d_Fecha
        df_RD_NoMapeo=pd.concat([df_RD_NoMapeo,df_Aux])

        # Determinar al aperturas únicas por día
        df_mmtodayA=df_mmtoday[(df_mmtoday.TipoAfe!='RD')]
        df_Ap_dia=get_aper_dia(df_mmtodayA)

        df_Ap_dia=pd.merge(df_Ap_dia,df_SIO_Dig, left_on='Elemento',right_on='Nombre_SIO', how='left')[['Elemento','TipoAfe','Pini','Pfin','Fkey','Estado','TipoElm']]
        df_Ap_dia['Incluir']=1

        # Validar las apertudas que no se encontraron en el MAPEO
        df_Aux=df_Ap_dia[df_Ap_dia['Fkey'].isna()][['Elemento']]
        df_Aux['Fecha']=d_Fecha
        df_AP_NoMapeo=pd.concat([df_AP_NoMapeo,df_Aux])

        # Exportar a excel la información de cada fecha con el detalle
        s_sheetname=str(d_Fecha.month) + '_' + str(d_Fecha.day)
        df_mmtoday.to_excel(writer, sheet_name=s_sheetname, index=False)
        worksheet = writer.sheets[s_sheetname]
        worksheet.set_column('A:A', 12)
        worksheet.set_column('B:B', 50)
        worksheet.set_column('C:D', 18)
        worksheet.set_column('E:E', 20)
        worksheet.set_column('F:F', 10)
        worksheet.set_column('G:G', 20)
        worksheet.set_column('H:I', 5)

        # Exportar a excel la información de aperturas día
        s_sheetname=str(d_Fecha.month) + '_' + str(d_Fecha.day) + 'A'
        df_Ap_dia.to_excel(writer, sheet_name=s_sheetname, index=False)
        worksheet = writer.sheets[s_sheetname]
        worksheet.set_column('A:A', 50)
        worksheet.set_column('B:B', 10)
        worksheet.set_column('C:D', 5)
        worksheet.set_column('E:G', 15)
        worksheet.set_column('H:H', 5)

        # Exportar a excel la información de cada fecha con el detalle
        s_sheetname=str(d_Fecha.month) + '_' + str(d_Fecha.day) + 'R'
        df_RD.to_excel(writer, sheet_name=s_sheetname, index=False)
        worksheet = writer.sheets[s_sheetname]
        worksheet.set_column('A:A', 12)
        worksheet.set_column('B:B', 50)
        worksheet.set_column('C:C', 8)
        worksheet.set_column('D:D', 15)
        worksheet.set_column('E:F', 5)
        worksheet.set_column('G:I', 15)
        worksheet.set_column('J:J', 5)



        d_Fecha=d_Fecha+delta




In [10]:
import tkinter as tk
from tkinter import messagebox
from plyer import notification

# Function to show a pop-up window
def show_message_box(title, message):
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    # Show the message box
    messagebox.showinfo(title, message)

if df_RD_NoMapeo.shape[0]>1 and df_AP_NoMapeo.shape[0]>1:
    s_msj='Falta mapeo en aperturas y en RD'
    print('RD')
    print(df_RD_NoMapeo)
    print('Aperturas')
    print(df_AP_NoMapeo)

elif df_RD_NoMapeo.shape[0]>1:
    s_msj='Falta mapeo en RD'
    print('RD')
    print(df_RD_NoMapeo)
elif df_AP_NoMapeo.shape[0]>1:
    s_msj='Falta mapeo en Aperturas'
    print('Aperturas')
    print(df_AP_NoMapeo)



# Function to show a notification
def show_notification(title, message):
    notification.notify(
        title=title,
        message=message,
        app_icon=None,  # e.g., 'C:\\path\\to\\icon_32x32.ico'
        timeout=30,  # seconds
    )

# Example usage
show_notification("Mensaje de Advertencia", s_msj)

# Example usage
show_message_box("Mensaje de Advertencia", s_msj)


RD
  Consecutivo                           Elemento      Fecha
4    C2005192  BL2 GUATIGUARA A SOCHAGOTA 230 kV 2024-02-27
5    C2005192   BL2 GUATIGUARA A PALENQUE 230 kV 2024-02-27


In [11]:
# import openpyxl

# s_file='Mmtosdia.xlsx'
# s_filepath=s_parentpath.joinpath(s_folderpath,str(year) + '-' + "{:02d}".format(month),s_file)

# excel_file=openpyxl.load_workbook(s_filepath)

# o_sheet=excel_file['1_29']

# cell_value=o_sheet.cell(row=1,column=1).value

# s_file='Consignaciones_Trans_' + str(year) + '_' + "{:02d}".format(month) + '.xlsx'
# fileDetalle=s_parentpath.joinpath(s_folderpath,str(year) + '-' + "{:02d}".format(month) + '\\DetalleMmto',s_file)
# o_sheet.cell(row=1, column=1).hyperlink = r'[' + str(fileDetalle) + r']C2011191!A1'

# #Save file
# excel_file.save(s_filepath)

# print(cell_value)